In [8]:
import pandas as pd
import json


df = pd.read_excel("validate.xlsx")
print(df.head())

from analysisrag import get_vulnerabilities

for i in range(len(df)):
    code = df.iloc[i]["code"]
    vulnerabilities = get_vulnerabilities(code)
    df.at[i, "vulnerabilities"] = json.dumps(vulnerabilities)

                                                code  \
0  // SPDX-License-Identifier: MIT\npragma solidi...   
1  // SPDX-License-Identifier: MIT\npragma solidi...   
2  // SPDX-License-Identifier: MIT\npragma solidi...   

                                             contain  \
0  [{\n    "line": "12",\n    "token": "payable(m...   
1                                                 []   
2  [{\n    "line": "8",\n    "token": "balances[m...   

                                                miss  
0                                                 []  
1  [{\n    "line": "12",\n    "token": "payable(m...  
2                                                 []  
[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract PiggyBank{\n    mapping(address => uint) public credit;\n    \n    function deposit() public payable {\n        credit[msg.sender] += msg.value;\n    }\n\n    function withdraw(uint 

In [16]:
errors = 0
hits = 0

for i in range(len(df)):
    v = df.iloc[i]["vulnerabilities"] 
    c = df.iloc[i]["contain"] 
    m = df.iloc[i]["miss"] 
    v = json.loads(v)
    v = [i for i in v if i["severity"] in ["high", "medium"]]

    problems = [i["problem"] for i in v]
    lines = [i["line"] for i in v]
    tokens = [i["token"] for i in v]

    # print("problems: ", problems)
    # print("lines: ", lines)
    # print("tokens: ", tokens)

    for e in json.loads(m):
        if e["line"] in lines:
            errors += 1
        else: 
            hits += 1

        if e["problem"] in problems:
            errors += 1
        else: 
            hits += 1

        if e["token"] in tokens:
            errors += 1
        else: 
            hits += 1

    for e in json.loads(c):
        if e["line"] in lines:
            hits += 1
        else: 
            errors += 1

        if e["problem"] in problems:
            hits += 1
        else: 
            errors += 1

        if e["token"] in tokens:
            hits += 1
        else: 
            errors += 1

print("accuracy: ", hits/(hits+errors))       


accuracy:  0.4444444444444444
